In [11]:
import os
import re, nltk
import PyPDF2
import pickle
import pdftotext
import string
import sys
import fileinput
from pathlib import Path
from nltk import word_tokenize

# Extract the data from the pdf

In [12]:
#read folder into folder
def read_pdf(file_path):
    #     file_path = str(input("Enter training directory path: \n"))
    files_list = []
    # PDF documents filepath
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if file.endswith(".pdf"):  # read all file end with pdf
                # store the read file name into a list
                files_list.append(os.path.join(root, file))
    return files_list


In [13]:
def store_folder_file(parent_dir, directory,  file_name, pdf):
    path = os.path.join(parent_dir, directory)
    if not os.path.exists(path):
        os.mkdir(path)
#     print("Directory '% s' created" % path)
#     print("file_name '% s' created" % file_name)
#     file_name = head_list[8]
#     new_files_path = path + "/"+ file_name +'.file'
    new_files_path = path + "/" + file_name + '.txt'
    # Save all text to a txt file.
    with open(new_files_path, 'w', encoding='utf-8') as f:
        f.write("\n\n".join(pdf))
        print("Files '% s' created" % file_name)


In [14]:
# Extracts the text from all pdf files in a specified directory
def pdf_to_txt(folder, new_folder):
    
    for file in read_pdf(folder):
        #     print(files_list)
        content_data = ""
        # Load your PDF, extract the content from pdf
        # Do data cleaning here
        with open(file, "rb") as f:
            pdf = pdftotext.PDF(f)
        head, tail = os.path.splitext(file)
        head_list = head.split(os.sep)
        store_folder_file(new_folder, head_list[7], head_list[8], pdf)
        # print(f"{new_folder}, new_folder_type = {type(new_folder)}, {dir}, {file}")


In [15]:
# experiment
# Extracts the text from all pdf files in a specified directory
def pdf_to_txt_exp(folder):

    for file in read_pdf(folder):
        #     print(files_list)
        content_data = ""
        # Load your PDF, extract the content from pdf
        with open(file, "rb") as f:
            pdf = pdftotext.PDF(f)


# Text processing

if the tasks require full sentence structure, like next word prediction or grammar check, this step can be skipped

In [16]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import numpy as np
nltk.download('punkt')  # it is one time execution
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

def tokenize_remove_stp(text):
    try:
       word_tokens = word_tokenize(text)
       filtered_word = [w for w in word_tokens if not w in stop_words]
       filtered_word = [w + " " for w in filtered_word]
       return filtered_word
    except:
       return np.nan


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\heihe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\heihe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\heihe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# Condenses all repeating newline characters into one single newline character
def text_processing(text):
    #removing extra line
    content = '\n'.join([p for p in re.split('\n|\r', text) if len(p) > 0])

    #case normalization & removing punctuations using set
    content = content.lower()
    # trivial way of regex for punctuation
    p_punct = re.compile(f"[{string.punctuation}]")
    content = re.sub(p_punct, "", content)

    # content = ''.join([i.lower()
    #                   for i in content if i not in string.punctuation])

    #tokenize and remove the stopwords
    content_new = tokenize_remove_stp(content)

    #Lemmatize the text
    lemmatizer = WordNetLemmatizer()
    word_lemm = [lemmatizer.lemmatize(word) for word in content_new]
    join_word_lemm = " ".join(word_lemm)
    return join_word_lemm


In [18]:
def read_write_text_file(file_path):
    try:
        with open(file_path, 'r+', encoding="utf-8") as f:
            text = f.read()
        with open(file_path, "w+", encoding="utf-8") as f:
            f.write(text_processing(text))
        print("Cleaned text", file_path)
    except FileNotFoundError:
        print("File not found")

In [19]:
def read_doc(path):
    #     file_path = str(input("Enter training directory path: \n"))
    # PDF documents filepath
    for root, dirs, files in os.walk(path):
        for file in files:
            # read all file end with pdf
            if file.endswith(".txt"):
                # store the read file name into a list
                file_path = f"{root}\{file}"
                read_write_text_file(file_path)


folder = r'C:\Users\heihe\Desktop\Malaysia-Legal-Doc-Classification\code_revised\combinned_data_txt'
print (read_doc(folder))

In [20]:
if __name__=="__main__":
    training_files_path = str(
        input("Enter training corpus directory path: \n"))
    testing_files_path = str(
        input("Enter testing corpus directory path: \n"))
    training_head_collection = []
    testing_head_collection = []

    #Training data set
    #get the correct index number to create correct file name
    for file in read_pdf(training_files_path):
        head, tail = os.path.splitext(file)
        training_head_collection.append(head)

    head_list = training_head_collection[0].split(os.sep)
    print(head_list)
    for i in range(len(head_list)):
        print(f"{i}= {head_list[i]}")

    training_data_dir = str(input("Training directory with folder name to content txt: "))
    
    #create the folder if the folder does not exit
    Path(training_data_dir).mkdir(parents=True, exist_ok=True)
    pdf_to_txt(training_files_path, training_data_dir)

    #do the data processing of the text file
    print(read_doc(training_data_dir))

    #Testing data set
    for file in read_pdf(testing_files_path):
        head, tail = os.path.splitext(file)
        testing_head_collection.append(head)

    head_list = testing_head_collection[0].split(os.sep)
    print(head_list)
    for i in range(len(head_list)):
        print(f"{i}= {head_list[i]}")

    testing_data_dir = str(
        input("Testing directory with folder name to content txt: "))

    #create the folder if the folder does not exit
    Path(testing_data_dir).mkdir(parents=True, exist_ok=True)
    pdf_to_txt(testing_files_path, testing_data_dir)

    #do the data processing of the text file
    print(read_doc(testing_data_dir))


['C:', 'Users', 'heihe', 'Desktop', 'Malaysia-Legal-Doc-Classification', 'code_revised', 'training_data_pdf', 'Abetment', 'CLJ_2003_4_409_psb']
0= C:
1= Users
2= heihe
3= Desktop
4= Malaysia-Legal-Doc-Classification
5= code_revised
6= training_data_pdf
7= Abetment
8= CLJ_2003_4_409_psb
Files 'CLJ_2003_4_409_psb' created
Files 'CLJ_2010_2_1_psb' created
Files 'LNS_2017_1_781_psb' created
Files 'CLJ_2019_4_705_psb' created
Files 'CLJ_2019_4_723_psb' created
Files 'CLJ_2019_4_799_psb' created
Files 'CLJ_2019_5_23_psb' created
Files 'CLJ_2019_5_293_psb' created
Files 'CLJ_2019_5_355_psb' created
Files 'CLJ_2019_5_93_psb' created
Files 'CLJ_2019_6_561_psb' created
Files 'CLJ_2015_8_329_psb' created
Files 'CLJ_2018_3_662_psb' created
Files 'CLJ_2018_5_326_psb' created
Files 'LNS_2015_1_1208_psb' created
Files 'LNS_2015_1_668_psb' created
Files 'LNS_2017_1_164_psb' created
Files 'LNS_2017_1_1804_psb' created
Files 'CLJ_2017_10_1_psb' created
Files 'CLJ_2018_4_236_psb' created
Files 'CLJ_2019_